# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [71]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
from pathlib import Path
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [72]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jayapura,-2.5333,140.7000,32.66,69,24,3.20,ID,1727503487
1,1,port-aux-francais,-49.3500,70.2167,2.11,70,42,13.06,TF,1727503488
2,2,hamilton,39.1834,-84.5333,20.56,86,100,5.14,US,1727503489
3,3,porto novo,6.4965,2.6036,24.02,93,88,1.75,BJ,1727503490
4,4,dzilam gonzalez,21.2833,-88.9333,24.49,88,89,1.13,MX,1727503492


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [73]:
%%capture --no-display

# Configure the map plot
map_humid = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
    )


# Display the map
map_humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [77]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_df = city_data_df.loc[(city_data_df["Max Temp"]>20) & (city_data_df["Max Temp"] <=32) & (city_data_df["Humidity"] < 60) & (city_data_df["Cloudiness"] ==0),:]
perfect_df

# Drop any rows with null values
# I want cloudiness to be 0; therefore no values dropped.

# Display sample data
perfect_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
134,134,nava,28.4167,-100.7667,24.57,40,0,0.62,MX,1727503665
147,147,ben gardane,33.1378,11.2197,28.05,42,0,0.93,TN,1727503681
156,156,socorro,31.6546,-106.3033,28.60,22,0,1.54,US,1727503692
184,184,abnub,27.2696,31.1511,26.61,47,0,4.81,EG,1727503731
245,245,crane,31.3974,-102.3501,24.17,39,0,3.43,US,1727503812
253,253,del rio,29.3627,-100.8968,24.06,50,0,3.09,US,1727503822
261,261,newman,37.3138,-121.0208,21.83,53,0,0.89,US,1727503834
338,338,tamanrasset,22.7850,5.5228,24.95,29,0,3.60,DZ,1727503937
349,349,nishon tumani,38.6940,65.6751,23.14,38,0,4.12,UZ,1727503951
369,369,sinop,41.7711,34.8709,21.86,53,0,2.35,TR,1727503977


### Step 3: Create a new DataFrame called `hotel_df`.

In [78]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data

hotel_df = perfect_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df




,City,Country,Lat,Lng,Humidity,Hotel Name
134,nava,MX,28.4167,-100.7667,40,
147,ben gardane,TN,33.1378,11.2197,42,
156,socorro,US,31.6546,-106.3033,22,
184,abnub,EG,27.2696,31.1511,47,
245,crane,US,31.3974,-102.3501,39,
253,del rio,US,29.3627,-100.8968,50,
261,newman,US,37.3138,-121.0208,53,
338,tamanrasset,DZ,22.7850,5.5228,29,
349,nishon tumani,UZ,38.6940,65.6751,38,
369,sinop,TR,41.7711,34.8709,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [81]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "apiKey":geoapify_key,  
    "categories": "accommodation.hotel"
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nava - nearest hotel: No hotel found
ben gardane - nearest hotel: No hotel found
socorro - nearest hotel: WoodSpring Suites El Paso I-10 Southeast
abnub - nearest hotel: University Guest House
crane - nearest hotel: No hotel found
del rio - nearest hotel: La Quinta Inn Del Rio
newman - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
nishon tumani - nearest hotel: Жура ота
sinop - nearest hotel: No hotel found
hurghada - nearest hotel: Sea Horse Hotel
al khums - nearest hotel: فندق الخمس
misratah - nearest hotel: الفندق السياحى
antonio enes - nearest hotel: Hotel Quirimbas
al jawf - nearest hotel: No hotel found
nampula - nearest hotel: Hotel Lurio
khash - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
chasong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
134,nava,MX,28.4167,-100.7667,40,No hotel found
147,ben gardane,TN,33.1378,11.2197,42,No hotel found
156,socorro,US,31.6546,-106.3033,22,WoodSpring Suites El Paso I-10 Southeast
184,abnub,EG,27.2696,31.1511,47,University Guest House
245,crane,US,31.3974,-102.3501,39,No hotel found
253,del rio,US,29.3627,-100.8968,50,La Quinta Inn Del Rio
261,newman,US,37.3138,-121.0208,53,No hotel found
338,tamanrasset,DZ,22.7850,5.5228,29,فندق الأمان
349,nishon tumani,UZ,38.6940,65.6751,38,Жура ота
369,sinop,TR,41.7711,34.8709,53,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)